# Prophet Model for Anomaly Detection

This notebook demonstrates how to use the Prophet model to detect anomalies in time series data, specifically API volume data with anomalies.

In [ ]:
# Install necessary packages
!pip install pandas numpy matplotlib prophet

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

### Step 1: Load the Dataset

In [ ]:
# Load the dataset
file_path = '/content/api_volume_data_with_anomalies_rounded.csv'  # Adjust this path if needed
data = pd.read_csv(file_path)
data.head()

### Step 2: Prepare Data for Prophet

In [ ]:
# Rename columns to fit Prophet's expected format
data.rename(columns={'time': 'ds', 'login_count': 'y'}, inplace=True)
data.head()

### Step 3: Initialize and Fit the Prophet Model

In [ ]:
# Initialize the model
model = Prophet()
model.fit(data)

### Step 4: Make Predictions

In [ ]:
# Create a dataframe for predictions
future = model.make_future_dataframe(periods=0, freq='H')  # Only using existing data for anomaly detection
forecast = model.predict(future)

### Step 5: Identify Anomalies

In [ ]:
# Add actual data for comparison
forecast['actual'] = data['y'].values

# Calculate anomalies: When actual values fall outside the confidence interval
forecast['anomaly'] = (forecast['actual'] > forecast['yhat_upper']) | (forecast['actual'] < forecast['yhat_lower'])
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'actual', 'anomaly']].head()

### Step 6: Plot Results with Anomalies

In [ ]:
# Plot actual data with Prophet's forecast and confidence intervals
plt.figure(figsize=(14, 7))
plt.plot(forecast['ds'], forecast['actual'], label='Actual Data', color='blue')
plt.plot(forecast['ds'], forecast['yhat'], label='Forecast', color='green')
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='lightgreen', alpha=0.5)

# Highlight anomalies
anomalies = forecast[forecast['anomaly']]
plt.scatter(anomalies['ds'], anomalies['actual'], color='red', label='Anomalies', marker='o')

plt.xlabel('Date')
plt.ylabel('Login Count')
plt.title('API Volume Anomaly Detection with Prophet')
plt.legend()
plt.show()

### Step 7: Save the Results

In [ ]:
# Save results with anomaly indicators
output_path = '/content/prophet_anomaly_detection_output.csv'  # Adjust path as needed
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'actual', 'anomaly']].to_csv(output_path, index=False)
print(f'Results saved to {output_path}')